<a href="https://colab.research.google.com/github/AlexandreCandidoXavier/BR-DWGD/blob/main/exemplos/exemplo_google_earth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Exemplo abrindo precipitaçao com Google Earth Engine e plotando para uma posição a precipitação
Para rodar será necessário fazer uma cópia no seu Google Drive. Menu File -> Save copy in Drive.

In [1]:
# instalando geemap atualizado (https://geemap.org/)
# se estiver no jupyterlab rodar:
%pip install -U "geemap[workshop]" -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 64.8 MB/s eta 0:00:00


In [2]:
import ee
import geemap
import numpy as np
import datetime
import pandas as pd
import plotly.express as px

In [24]:
# vai pedir a autentificação (tem que ter conta gmail)
m = geemap.Map()

In [25]:
# pegando variavel precipitação
# para as demais ver: https://gee-community-catalog.org/projects/br_dwgd/
var = ee.ImageCollection("projects/sat-io/open-datasets/BR-DWGD/PR")

# para reescalonar
offset = 225
scale  = 0.006866665

# reescalonando
def scaleBand(image):
    scaledImage = image.select("b1").rename(['pr']).multiply(scale).add(offset)
    return scaledImage.copyProperties(image, image.propertyNames())

var = var.map(scaleBand)
color_pal = ['000066', '001199', '0044BB', '0077DD', '33AAEE', '66CCFF', 'FFDDCC', 'FFBB99', 'FF9966', 'FF6644']
vis = {'min': 0,'max': 70, 'palette': color_pal}

# definindo período (tem limite)
start_date, end_date = '1980-01-01','2023-01-01' #'2010-01-01','2023-01-01'
var = var.filterDate(start_date, end_date).select(['pr']);
m.addLayer(var.first(), vis, 'Precipitação')

In [29]:
# coordenada do ponto de interesse [lon, lat]
lon_lat = [-43.04, -14.33]
point = {'type':'Point', 'coordinates': lon_lat}
m.addLayer(ee.Feature(point), {'color': 'red'}, "posição")

In [27]:
# plotando a precipitação da data inicial
m.centerObject(var.first())
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

Pegando infromações para uma posição

In [30]:
info = var.getRegion(point, 500).getInfo()

Plotando dados diários para o período

In [52]:
header = info[0]
data = np.array(info[1:])
iTime = header.index('time')
time = [datetime.datetime.fromtimestamp(i/1000) for i in (data[0:,iTime].astype(int))]

#criando dataframe
prec = data[:,-1].astype('float')
prec[prec<0] = 0
df = pd.DataFrame(prec, columns=['Precipitação (mm)'])
df.set_index(np.array(time), inplace=True)
df.index.name='Data'
# plotando
fig = px.line(df, x=df.index, y='Precipitação (mm)',
                 title=f"Precipitação na lat={lon_lat[1]}; lon={lon_lat[0]}")
fig.show()

Para salvar em formato "csv"

In [45]:
# df.to_csv('dados.csv')

Agregando dados diários para mensal

In [53]:
df_mes = df.resample('M').sum()
df_mes

,Precipitação (mm)
Data,
1980-01-31,365.203265
1980-02-29,233.006242
1980-03-31,18.471196
1980-04-30,27.603933
1980-05-31,0.000000
...,...
2022-08-31,0.000000
2022-09-30,19.363983
2022-10-31,11.556573


In [54]:
# Plotando mensal
fig = px.line(df_mes, x=df_mes.index, y='Precipitação (mm)',
                 title=f"Precipitação mensal na lat={lon_lat[1]}; lon={lon_lat[0]}")
fig.update_traces(hovertemplate="Data: %{x} <br>prec: %{y:.1f} mm")
fig.show()

In [55]:
df_ano = df.resample('Y').sum()
fig = px.bar(df_ano, x=df_ano.index, y='Precipitação (mm)')
fig.update_traces(hovertemplate="Data: %{x} <br>prec: %{y:.0f} mm")
fig.show()